In [1]:
# PyData
import pandas
import numpy
from pandas import DataFrame, Series
from sklearn import datasets, linear_model, preprocessing, cross_validation
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge

# System
import datetime
import os
import math

# Graphing
%matplotlib inline
import seaborn


In [2]:
df_basic = DataFrame.from_csv("train_features_basic.csv", index_col=['Date', 'Store'])
df_means = DataFrame.from_csv("train-features-predicted_mean.csv", index_col=['Date', 'Store'])
del df_means['Sales-prediction_mean_error'] # Not supported

In [3]:
df_sales = DataFrame.from_csv("train.csv", index_col=['Date', 'Store'])[['Sales']]

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
df_features = df_means.join(df_basic, how='outer')
train_df = df_sales.join(df_features)
train_df['Sales_predicted'] = train_df['Sales_predicted'].fillna(0) # when sales is 0, sales_predicted is nan.
train_df.head()

Sales  Sales_predicted  DayOfWeek  Open  Promo  \
Date       Store                                                   
2015-07-31 1       5263      5145.283582          5     1      1   
           2       6064      5115.882353          5     1      1   
           3       8314      8138.089552          5     1      1   
           4      13995     10275.776119          5     1      1   
           5       4822      5308.835821          5     1      1   

                  StateHoliday  SchoolHoliday  cal:dayOfMonth  cal:dayOfWeek  \
Date       Store                                                               
2015-07-31 1                 0              1              31              4   
           2                 0              1              31              4   
           3                 0              1              31              4   
           4                 0              1              31              4   
           5                 0              1              31              4   

                  cal:dayofyear  cal:month  cal:quarter  cal:weekofyear  \
Date       Store                                                          
2015-07-31 1                212          7            3              31   
           2                212          7            3              31   
           3                212          7            3              31   
           4                212          7            3              31   
           5                212          7            3              31   

                  cal:year  
Date       Store            
2015-07-31 1          2015  
           2          2015  
           3          2015  
           4          2015  
           5          2015

In [5]:
del df_features
del df_basic
del df_means
del df_sales

In [6]:
train_df['Sales_predicted'] = train_df['Sales_predicted'].fillna(train_df['Sales_predicted'].mean())

In [7]:
train_df['Sales_predicted'] = train_df['Sales_predicted'].map(lambda x: int(x))

In [8]:
train_df.reset_index(inplace=True)

In [9]:
del train_df['Date']

In [10]:
feature_columns = list(train_df.columns)
feature_columns.remove("Sales")
feature_columns = ['Sales'] + feature_columns
train_df = train_df[feature_columns]

In [11]:

train_df.head()

,Sales,Store,Sales_predicted,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year
0,5263,1,5145,5,1,1,0,1,31,4,212,7,3,31,2015
1,6064,2,5115,5,1,1,0,1,31,4,212,7,3,31,2015
2,8314,3,8138,5,1,1,0,1,31,4,212,7,3,31,2015
3,13995,4,10275,5,1,1,0,1,31,4,212,7,3,31,2015
4,4822,5,5308,5,1,1,0,1,31,4,212,7,3,31,2015
